<a href="https://colab.research.google.com/github/Deepchavda007/Linear-regression-using-PyTorch/blob/main/Linear_regression_using_PyTorch_built_ins.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Linear regression using PyTorch built-ins

We've implemented linear regression & gradient descent model using some basic tensor operations. However, since this is a common pattern in deep learning, PyTorch provides several built-in functions and classes to make it easy to create and train models with just a few lines of code.

Let's begin by importing the `torch.nn` package from PyTorch, which contains utility classes for building neural networks.

In [ ]:
import torch.nn as nn
import numpy as np
import torch as t

In [ ]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70], 
                   [74, 66, 43], 
                   [91, 87, 65], 
                   [88, 134, 59], 
                   [101, 44, 37], 
                   [68, 96, 71], 
                   [73, 66, 44], 
                   [92, 87, 64], 
                   [87, 135, 57], 
                   [103, 43, 36], 
                   [68, 97, 70]], 
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119],
                    [57, 69], 
                    [80, 102], 
                    [118, 132], 
                    [21, 38], 
                    [104, 118], 
                    [57, 69], 
                    [82, 100], 
                    [118, 134], 
                    [20, 38], 
                    [102, 120]], 
                   dtype='float32')

inputs = t.from_numpy(inputs)
targets = t.from_numpy(targets)

## Dataset and DataLoader

We'll create a `TensorDataset`, which allows access to rows from `inputs` and `targets` as tuples, and provides standard APIs for working with many different types of datasets in PyTorch.

In [ ]:
from torch.utils.data import TensorDataset

In [ ]:
# define datasets
train_ds = TensorDataset(inputs,targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]), tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

We'll also create a `DataLoader`, which can split the data into batches of a predefined size while training. It also provides other utilities like shuffling and random sampling of the data.

In [ ]:
from torch.utils.data import DataLoader

In [ ]:
batch_size = 5
train_dl = DataLoader(train_ds,batch_size,shuffle=True)

In [ ]:
# We can use the data loader in a for loop. Let's look at an example

for xb,yb in train_dl:
    print(xb)
    print(yb)
    break

tensor([[73., 67., 43.],
        [68., 96., 71.],
        [74., 66., 43.],
        [68., 97., 70.],
        [69., 96., 70.]])
tensor([[ 56.,  70.],
        [104., 118.],
        [ 57.,  69.],
        [102., 120.],
        [103., 119.]])


If `shuffle` is set to `True`, it shuffles the training data before creating batches. Shuffling helps randomize the input to the optimization algorithm, leading to a faster reduction in the loss.

## nn.Linear

Instead of initializing the weights & biases manually, we can define the model using the `nn.Linear` class from PyTorch, which does it automatically.

In [ ]:
# Define model

model = nn.Linear(3,2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[-0.5047, -0.5168,  0.0841],
        [-0.2725,  0.5279, -0.2256]], requires_grad=True)
Parameter containing:
tensor([-0.1164,  0.2040], requires_grad=True)


PyTorch models also have a helpful `.parameters` method, which returns a list containing all the weights and bias matrices present in the model. For our linear regression model, we have one weight matrix and one bias matrix.

In [ ]:
# Parameters

list(model.parameters())

[Parameter containing:
 tensor([[-0.5047, -0.5168,  0.0841],
         [-0.2725,  0.5279, -0.2256]], requires_grad=True),
 Parameter containing:
 tensor([-0.1164,  0.2040], requires_grad=True)]

In [ ]:
preds = model(inputs)
preds

tensor([[ -67.9640,    5.9799],
        [ -86.1340,    7.4235],
        [-108.3908,   34.1525],
        [ -70.7016,  -13.2411],
        [ -78.6609,   16.2897],
        [ -67.9519,    5.1794],
        [ -85.5332,    6.6700],
        [-108.8114,   33.6544],
        [ -70.7137,  -12.4406],
        [ -78.0721,   16.3366],
        [ -67.3631,    5.2264],
        [ -86.1219,    6.6231],
        [-108.9917,   34.9060],
        [ -71.2904,  -13.2881],
        [ -78.6730,   17.0901]], grad_fn=<AddmmBackward0>)

## Loss Function

* Instead of defining a loss function manually, we can use the built-in loss function `mse_loss`.

* The `nn.functional` package contains many useful loss functions and several other utilities.
 

In [ ]:
import torch.nn.functional as F

In [ ]:
# Define the loss function
loss_fn = F.mse_loss

In [ ]:
# Let's compute the loss for the current predictions of our model
loss = loss_fn(preds,targets)
loss

tensor(17203.0957, grad_fn=<MseLossBackward0>)

## Optimizer

* Instead of manually manipulating the model's weights & biases using gradients, we can use the optimizer `optim.SGD`. SGD is short for "stochastic gradient descent". The term _stochastic_ indicates that samples are selected in random batches instead of as a single group.


* Note that `model.parameters()` is passed as an argument to `optim.SGD` so that the optimizer knows which matrices should be modified during the update step. Also, we can specify a learning rate that controls the amount by which the parameters are modified.

In [ ]:
# Define optimizer
opt = t.optim.SGD(model.parameters(), lr=1e-5)

## Train the model

We are now ready to train the model. We'll follow the same process to implement gradient descent:

1. Generate predictions

2. Calculate the loss

3. Compute gradients w.r.t the weights and biases

4. Adjust the weights by subtracting a small quantity proportional to the gradient

5. Reset the gradients to zero

The only change is that we'll work batches of data instead of processing the entire training data in every iteration. Let's define a utility function `fit` that trains the model for a given number of epochs.

In [ ]:
# Utility function to train the model
def fit(num_epochs, model, loss_fn, opt, train_dl):
    
    # Repeat for given number of epochs
    for epoch in range(num_epochs):
        
        # Train with batches of data
        for xb,yb in train_dl:
            
            # 1. Generate predictions
            pred = model(xb)
            
            # 2. Calculate loss
            loss = loss_fn(pred, yb)
            
            # 3. Compute gradients
            loss.backward()
            
            # 4. Update parameters using gradients
            opt.step()
            
            # 5. Reset the gradients to zero
            opt.zero_grad()
        
        # Print the progress
        if (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

Some things to note above:

* We use the data loader defined earlier to get batches of data for every iteration.

* Instead of updating parameters (weights and biases) manually, we use `opt.step` to perform the update and `opt.zero_grad` to reset the gradients to zero.

* We've also added a log statement that prints the loss from the last batch of data for every 10th epoch to track training progress. `loss.item` returns the actual value stored in the loss tensor.



In [ ]:
fit(150,model,loss_fn,opt,train_dl)

Epoch [10/150], Loss: 186.0129
Epoch [20/150], Loss: 159.9290
Epoch [30/150], Loss: 118.5764
Epoch [40/150], Loss: 47.0545
Epoch [50/150], Loss: 82.2243
Epoch [60/150], Loss: 56.2823
Epoch [70/150], Loss: 60.3355
Epoch [80/150], Loss: 48.8281
Epoch [90/150], Loss: 28.9398
Epoch [100/150], Loss: 34.4484
Epoch [110/150], Loss: 25.6473
Epoch [120/150], Loss: 26.5246
Epoch [130/150], Loss: 35.3602
Epoch [140/150], Loss: 26.9102
Epoch [150/150], Loss: 16.2935


In [ ]:
# Generate predictions
preds = model(inputs)
preds

tensor([[ 57.3191,  71.1391],
        [ 82.1265,  96.7869],
        [117.3750, 141.0075],
        [ 23.0612,  40.7520],
        [100.6939, 110.2027],
        [ 56.1346,  69.9934],
        [ 81.9742,  96.1483],
        [117.6824, 141.2126],
        [ 24.2457,  41.8977],
        [101.7260, 110.7098],
        [ 57.1667,  70.5005],
        [ 80.9421,  95.6412],
        [117.5273, 141.6460],
        [ 22.0291,  40.2449],
        [101.8784, 111.3483]], grad_fn=<AddmmBackward0>)

In [ ]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 57.,  69.],
        [ 80., 102.],
        [118., 132.],
        [ 21.,  38.],
        [104., 118.],
        [ 57.,  69.],
        [ 82., 100.],
        [118., 134.],
        [ 20.,  38.],
        [102., 120.]])

In [ ]:
model(t.tensor([[75, 63, 44.]]))

tensor([[53.9756, 67.2146]], grad_fn=<AddmmBackward0>)